# Install PyCUDA

In [ ]:
!pip install pycuda

# Import libs and launch test code

In [ ]:
import pycuda.autoinit
import pycuda.driver as cuda
import numpy as np
from pycuda.compiler import SourceModule
import time


In [ ]:
mod = SourceModule("""
__global__ void multiply_them(float *dest, float *a, float *b)
{
  const int i = threadIdx.x;
  dest[i] = a[i] * b[i];
}
""")

a = np.random.randn(400).astype(np.float32)
b = np.random.randn(400).astype(np.float32)

# Allocate memory on the device
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
dest_gpu = cuda.mem_alloc(a.nbytes)

# Copy data from host to device
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)

# Execute the kernel
func = mod.get_function("multiply_them")
func(dest_gpu, a_gpu, b_gpu, block=(400,1,1))

# Copy result from device to host
dest = np.empty_like(a)
cuda.memcpy_dtoh(dest, dest_gpu)

# Display results
print(dest)


# Mandelbrot set: CUDA version

In [ ]:
mod = SourceModule("""
__device__ int mandelbrot(float real, float imag, int max_iter) {
    float r = real;
    float i = imag;
    for (int t = 0; t < max_iter; t++) {
        if (r * r + i * i > 4.0) return t;
        float new_r = r * r - i * i + real;
        i = 0;  // TODO: change 0 to correct code
        r = new_r;
    }
    return 0;
}


__global__ void compute_mandelbrot(float *output, int width, int height, float x_min, float x_max, float y_min, float y_max, int max_iter) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int idy = blockIdx.y * blockDim.y + threadIdx.y;
    if (idx < width && idy < height) {
        float real = x_min + idx * (x_max - x_min) / width;
        float imag = 0;  // TODO: change 0 to correct code
        output[idy * width + idx] = mandelbrot(real, imag, max_iter);
    }
}
""")

def cuda_mandelbrot(width, height, x_min, x_max, y_min, y_max, max_iter):
    output = np.zeros((height, width), dtype=np.float32)

    # Allocate memory on the device
    output_gpu = cuda.mem_alloc(output.nbytes)

    # Execute the kernel
    block_size = (16, 16, 1)
    grid_size = (width // block_size[0] + 1, height // block_size[1] + 1)

    start = time.time()
    func = mod.get_function("compute_mandelbrot")
    func(output_gpu, np.int32(width), np.int32(height), np.float32(x_min), np.float32(x_max), np.float32(y_min), np.float32(y_max), np.int32(max_iter), block=block_size, grid=grid_size)
    cuda.Context.synchronize()  # Wait for the computation to finish
    end = time.time()

    # Copy the result back to the host
    cuda.memcpy_dtoh(output, output_gpu)

    return output, end - start

width, height = 1024, 768
x_min, x_max = -2.0, 1.0
y_min, y_max = -1.5, 1.5
max_iter = 128

mandelbrot_image, time_cuda = cuda_mandelbrot(width, height, x_min, x_max, y_min, y_max, max_iter)
print("CUDA Time:", time_cuda)


# Mandelbrot set: Numpy (cpu) version

In [ ]:
def numpy_mandelbrot(width, height, x_min, x_max, y_min, y_max, max_iter):
    output = np.zeros((height, width), dtype=np.float32)
    for y in range(height):
        imag = y_min + y * (y_max - y_min) / height
        for x in range(width):
            real = 0  # TODO: change 0 to correct code
            r, i = real, imag
            for t in range(max_iter):
                if r * r + i * i > 4.0:
                    output[y, x] = t
                    break
                r = 0  # TODO: change 0 to correct code
                i = 2 * r * i + imag

    return output

start = time.time()
mandelbrot_image_numpy = numpy_mandelbrot(width, height, x_min, x_max, y_min, y_max, max_iter)
end = time.time()
time_numpy = end - start
print("CPU Time:", time_numpy)


# Validate results

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(ncols=2, figsize=(24, 8))
axs[0].imshow(mandelbrot_image)
axs[0].set_title(f"CUDA. Time: {time_cuda}")
axs[1].imshow(mandelbrot_image_numpy)
axs[1].set_title(f"Numpy. Time: {time_numpy}")
